# Transformer Model
In this notebook, we will use a Transformer model trained on our labeled data to predict whether some test data contains suicidal ideology or not, then compare to the actual labels and measure our metrics.

In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

import utils as utils

from huggingface_hub import login
login()

In [13]:
train_path = 'train.csv'

df = pd.read_csv(train_path)
print(df.head())
df.describe()

                                                text              label
0  wanting to skip or postpone my exam my exam is...       self.Anxiety
1  Do other bipolar folks have problems with subs...       self.bipolar
2  Wanted to share some revelations I just had to...    self.depression
3  I feel deader than dead. I find that I don't h...  self.SuicideWatch
4  I'm pretty sure my friends suicidal what do I ...  self.SuicideWatch


,text,label
count,45706,45706
unique,45631,5
top,FEARLESS FRIDAYS MEGA THREAD. Here we discuss ...,self.depression
freq,15,15714


In [14]:
df.isna().sum()

text     0
label    0
dtype: int64

In [15]:
class SuicideDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text'].values,
    pd.factorize(df['label'])[0],
    test_size=0.2,
    random_state=42,
    stratify=pd.factorize(df['label'])[0]
)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")

train_dataset = SuicideDataset(X_train, y_train, tokenizer, max_length=128)
test_dataset = SuicideDataset(X_test, y_test, tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [18]:
model = AutoModel.from_pretrained("mental/mental-bert-base-uncased")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    all_preds = []
    all_labels = []

    model.eval()
    total_val_loss = 0
    for batch in test_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.last_hidden_state.mean(dim=1)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    results = {'precision': [], 'recall': [], 'f1': [], 'accuracy': []}
    results['precision'].append(precision)
    results['recall'].append(recall)
    results['f1'].append(f1)
    results['accuracy'].append(accuracy)

    utils.create_graph([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], results, f'Transformer ({epoch + 1})', f'TF{epoch + 1}.png')
